## Amazon SageMaker DICOM Training Overview

In this example we will demonstrate how to integrate the [MONAI](http://monai.io) framework into Amazon SageMaker, and give example code of MONAI pre-processing transforms and neural network (DenseNet) that you can use to train a medical image classification model using DICOM images directly.  

Please also visit [Build a medical image analysis pipeline on Amazon SageMaker using the MONAI framework](https://aws.amazon.com/blogs/industries/build-a-medical-image-analysis-pipeline-on-amazon-sagemaker-using-the-monai-framework/) for additional details on how to deploy the MONAI model, pipe input data from S3, and perform batch inferences using SageMaker batch transform.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

Sample dataset is obtained from this [source COVID-CT-MD](https://github.com/ShahinSHH/COVID-CT-MD). The total dataset contains volumetric chest CT scans (DICOM files) of 169 patients positive for COVID-19 infection, 60 patients with CAP (Community Acquired Pneumonia), and 76 normal patients. For this demo purpose, only 26 images are randomly selected. The selection and preprocessing are not included in this demo. 

## Introduction to MONAI

MONAI is a PyTorch-based, open-source framework for deep learning in healthcare imaging, part of PyTorch Ecosystem.

Medical imaging has some unique requirememts. 
- Biomedical applications have specific requirements
- Image modalities (MR, CT, US, etc.) require specific data processing
- Data formats (DICOM, NIfTI, etc.) are specific to medical applications and require special support
- Certain network architectures are designed for, or are highly suitable for, biomedical applications
- Data transforms specific to biomedical applications, and to image modalities, are very useful when pre-processing data, augmenting data during training, and post-processing

MONAI provides a framework of deep learning facilities and infrastructure to meet these needs in a flexible Pytorch-compatible way:

- Data loading and handling library for biomedical file types
- Large set of data transforms to process, regularize, and augment image date before, during, and after training
- Library of general-purpose network, metric, and loss function definitions implementing common architectures
- Set of ready-made components for training and inference to utilize computing infrastructure efficiently

There is a **rich set of transforms** in six categories: Crop & Pad, Intensity, IO, Post-processing, Spatial, and Utilities. For more details, please visit all the transforms in MONAI. It support both Dictionary and Array format data. Transforms are composed with `Compose` to create a sequence of operations.
You can also create custom transform if needed.

There is a feature called **Dataset Caching**. MONAI provides multi-thread versions of CacheDataset to accelerate data transformation and training by storing the intermediate outcomes. Enabling this feature could potentially give up to 10x training speedup.

## Install necessary libraries

In [ ]:
!pip install monai --no-cache-dir

In [ ]:
# install the dependencies needed by monai
!pip install 'monai[all]'

In [ ]:
!pip install -r ./code/requirements.txt

In [ ]:
!pip install --upgrade torch torchvision  ## upgrade torchvision to ensure consistent performance

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
import sagemaker 

sess = sagemaker.Session()
env_path = Path('.') / 'set.env'
load_dotenv(dotenv_path=env_path)

bucket = sess.default_bucket() ## you can replace with your bucket for the dataset
bucket_path=os.environ.get('BUCKET_PATH')
user=os.environ.get('DICOM_USER')
password = os.environ.get('DICOM_PASSWORD')
datadir = 'data'
print('Bucket: '+bucket)

## Upload training dataset in S3

For this demo, we only use 25 images for model training, which is already downloaded and saved in data folder 

+ *.dcm are the dicome images
+ manifest.json stores labels for each image

In [ ]:
image_file_list=os.listdir(datadir)


image_file_list = [x  for x in image_file_list if x.endswith('dcm') ]

## Preprocess the dataset and display them

In [ ]:
import monai
from monai.transforms import Compose, LoadImage, Resize, ScaleIntensity, ToTensor, SqueezeDim, RandRotate,RandFlip,RandZoom
import matplotlib.pyplot as plt
# define transform functions 
## preprocess the dataset before trainining using MONAI.  Based on img.shape, this is a channel last image
train_transforms = Compose([
    LoadImage(image_only=True),
    ScaleIntensity(),
    RandRotate(range_x=15, prob=0.5, keep_size=True),
    RandFlip(spatial_axis=0, prob=0.5),
    #RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5, keep_size=True),
    Resize(spatial_size=(512,-1)),
    ToTensor()
])
img = train_transforms(datadir+'/'+image_file_list[0])
img.shape ## check image size after preprocessing

In [ ]:
#Display sample of DICOM Images
inf_test = []
inf_test_label = []

trans = Compose([LoadImage(image_only=True), Resize(spatial_size=(512,-1))])
plt.subplots(2, 2, figsize=(8, 8))
for i in range(0,4):
    #s3.download_file(bucket, image_file_list[i], datadir+'/'+image_file_list[i])
    
    img = train_transforms(datadir+'/'+image_file_list[i])
    print(img.shape)
    plt.subplot(2, 2, i + 1)
    plt.xlabel(image_file_list[i])
    plt.imshow(img[:,:,0], cmap='gray')
    plt.title(image_file_list[i].split('-')[0])
    inf_test.append(datadir+'/'+image_file_list[i])
    inf_test_label.append(image_file_list[i].split('-')[0])
    
plt.tight_layout()
plt.show()

In [ ]:
datadir_test='test_data'

image_file_list_test=os.listdir(datadir_test)
image_file_list_test

## Data

### Create Sagemaker session and S3 location for DICOM dataset

In [ ]:
import sagemaker
from sagemaker.s3 import S3Downloader, S3Uploader
import os

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

key='CovidTraining'
path=os.path.join("s3://",bucket,key)

## IF UPLOAD THE DATA TO S3, DO THE FOLLOWING STEP. we may skip the step if the the data has already been uploaded
inputs = S3Uploader.upload(local_path=datadir, desired_s3_uri=path) 

print('input spec as an S3 path: {}'.format(inputs))

In [ ]:
# Upload test data to S3
testdir='test_data'
test_key='test_data'
test_data_path=os.path.join("s3://",bucket,test_key)
test= S3Uploader.upload(local_path=testdir, desired_s3_uri=test_data_path)

## Train Model
### Training

The ```train.py``` script provides all the code we need for training and hosting a SageMaker model (model_fn function to load a model). The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* SM_MODEL_DIR: A string representing the path to the directory to write model artifacts to. These artifacts are uploaded to S3 for model hosting.
* SM_NUM_GPUS: The number of gpus available in the current container.
* SM_CURRENT_HOST: The name of the current container on the container network.
* SM_HOSTS: JSON encoded list containing all the hosts .
Supposing one input channel, 'training', was used in the call to the PyTorch estimator's fit() method, the following will be set, following the format SM_CHANNEL_[channel_name]:

* SM_CHANNEL_TRAINING: A string representing the path to the directory containing data in the 'training' channel.
For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an argparse.ArgumentParser instance.

## Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure.  We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters.  In this case we are going to run our training job on ```ml.m5.2xlarge``` instance.  But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)).  The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the ```monai_dicom.py``` script above.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    source_dir='code',
                    role=role,
                    framework_version='1.5.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.m5.4xlarge',
                    hyperparameters={
                        'backend': 'gloo',
                        'epochs': 100,
                         'seed':123
                    })

After we've constructed our PyTorch object, we can fit it using the DICOM dataset we uploaded to S3.

In [ ]:
%%time
estimator.fit({'train': inputs})

## Deploy the endpoint with the following options

+ default inference with `numpy` as input

+ customized inference with `JSON` file pointing to the image file in S3 [./source/inference.py]

for further information, you may refer to [pytoch-inference-hander](https://github.com/aws/sagemaker-pytorch-inference-toolkit/blob/master/src/sagemaker_pytorch_serving_container/default_pytorch_inference_handler.py) 

### Option 1: synchronous inference directly with estimator

In [ ]:

predictor = estimator.deploy(initial_instance_count=1,entry_point='inference.py', instance_type='ml.m5.xlarge',serializer=sagemaker.serializers.JSONSerializer(),deserializer=sagemaker.deserializers.JSONDeserializer())

In [ ]:
%%time
payload={"bucket": bucket,
    "key":"test_data/normal-IM0062.dcm"}

predictor.predict(payload)

In [ ]:
payload2={"bucket": bucket,
    "key":"test_data/covid-IM0073.dcm"}

predictor.predict(payload2)

### Option 2: Deploy through model data in S3

In [ ]:
model_data=estimator.__dict__['output_path']+estimator.__dict__['_current_job_name']+'/output/model.tar.gz'

In [ ]:
model_data

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    entry_point="inference.py", ## inference code with customerization
    #source_dir="code",        ## folder with the inference code
    role=role,
    model_data=model_data,
    framework_version="1.5.0",
    py_version="py3",
)
predictor2 = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge',entry_point='inference.py',source_dir='code',
                            serializer=sagemaker.serializers.JSONSerializer(),deserializer=sagemaker.deserializers.JSONDeserializer())

In [ ]:
%%time
payload={"bucket": bucket,
    "key":"test_data/normal-IM0062.dcm"}

predictor2.predict(payload)

In [ ]:
%%time
payload2={"bucket": bucket,
    "key":"test_data/covid-IM0073.dcm"}

predictor2.predict(payload2)